In [ ]:
if __name__ == '__main__':
    import os
    import sys 
    os.chdir(os.path.join(os.getenv('HOME'), 'ASL'))
    sys.path.insert(0, os.path.join(os.getenv('HOME'), 'ASL'))
    sys.path.append(os.path.join(os.path.join(os.getenv('HOME'), 'ASL') + '/src'))

import os
import sys 
import numpy as np
import torch
from pseudo_label import DetectronHelper
import time
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"src"))
from PIL import Image
import argparse
from visu import Visualizer    
from pseudo_label import readImage
from pathlib import Path
from utils_asl import load_yaml
from torchvision import transforms as tf

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from utils_asl import label_to_png

from visu import Visualizer
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=True, num_classes=41)

# label = fsh.get_label( i1 )

class FastDataset(Dataset):
  def __init__(self, paths):
    self.paths = paths
    self.output_transform = tf.Compose([
      tf.Normalize([.485, .456, .406], [.229, .224, .225])
    ])
  def __len__(self):
    return len(self.paths)
  
  def __getitem__(self,index):
    p = self.paths[index] 
    i1 = readImage(p, H=640, W=1280, scale=False)
    h,w,c = i1.shape
    
    return torch.from_numpy( i1 ) , torch.tensor( index )


def label_generation(**kwargs):
    # idea:
    # load model .ckpt
    # set scenes to pred
    # store as png to fil 
    env_cfg_path = os.path.join('cfg/env', os.environ['ENV_WORKSTATION_NAME']+ '.yml')
    env = load_yaml(env_cfg_path)
    idfs = str( kwargs["identifier"])
    idfs = "detectron_test_new_format"
    
    confidence = kwargs["confidence"]
    scenes = kwargs["scenes"]
    exp = kwargs["exp"]


    if os.environ['ENV_WORKSTATION_NAME'] == 'ptarmigan':
      base = os.path.join(env['scannet'], 'scans')
      scratchdir = "/media/scratch1/jonfrey/labels_generated"
    elif os.environ['ENV_WORKSTATION_NAME'] == 'ws': 
      base = os.path.join(env['scannet'], 'scans')
      scratchdir = "/home/jonfrey/Datasets/labels_generated"
    else:      
      scratchdir = os.getenv('TMPDIR')
      base = os.path.join(scratchdir, 'scannet', 'scans')


    fsh = DetectronHelper(device='cuda')
    export = os.path.join(scratchdir, idfs )
    paths = [str(s) for s in Path(base).rglob('*.jpg') if str(s).find("color") != -1]
    # filter to get evey 10 image
    paths = [s for s in paths if int(s.split('/')[-1][:-4]) % 10 == 0]
    
    # filter only scenes of interrest
    pa = []
    for scene in scenes:
      pa += [s for s in paths if s.find(scene) != -1]

    dataset = FastDataset( pa)
    dataloader = DataLoader(dataset,
      shuffle = False,
      num_workers = 4,
      pin_memory = False,
      batch_size = 1)
    
    h,w,_= readImage(pa[0], H=640, W=1280, scale=False).shape
    st = time.time()
    for j, batch in enumerate( dataloader ):
        print(j,"/" ,len(dataloader), " ", time.time()-st)
        img = np.uint8( batch[0].numpy()[0] )
        index = int(batch[1])
        
        _st = time.time()
        label_probs = torch.from_numpy( fsh.get_label_prob( img ) )
        print("Time: " , time.time()-_st)
        
        torch.cuda.empty_cache()
        label_probs = torch.nn.functional.interpolate( label_probs.type(torch.float32)[None] , (h,w), mode='bilinear')[0]
        l = label_probs
        l = l.permute( (1,2,0) )

        outpath = pa[index]
        outpath = outpath.replace("color", idfs)[:-4]+'.png'
        res = outpath[outpath.find('scans/'):]
        res = os.path.join(export, res)
        Path(res).parent.mkdir(exist_ok=True, parents= True)
        label_to_png( l[:,:,1:], res )
        

exp = load_yaml('/home/jonfrey/ASL/cfg/exp/create_newlabels/debug.yml')
label_generation(
 **exp["label_generation"],
 exp = exp
)
